# Parser

Code to take some files in the output format of the URL scraper, and translate them to the input format of the Python ML models

In [1]:
import json
import random
from collections import defaultdict

In [2]:
FILES = ["data_sanitized.json"]
OUTPUT = "sanitized_data_to_label.json"

In [6]:
random.seed(42)

result = {"data": []}

exists = {}
keys = defaultdict(None)
with open('data_to_label.json') as file:
    exists = json.load(file)
    keys = defaultdict(lambda: None, {entry['text']: entry for entry in exists['data']})

for file in FILES:
    with open(file) as f:
        entries = json.load(f)
        random.shuffle(entries)
        for entry in entries:
            existing_entry = keys[entry['main_heading']]
            if existing_entry:
                existing_entry.pop('objective', None)
                result['data'].append(existing_entry)
            else: 
                result["data"].append(
                    {
                        "text": entry["main_heading"],
                        "agency": False,
                        "humanComparison": False,
                        "hyperbole": False,
                        "uncriticalHistoryComparison": False,
                        "unjustifiedClaimsAboutFuture": False,
                        "deepSounding": False,
                        "sceptics": False,
                        "deEmphasize": False,
                        "performanceNumber": False,
                        "inscrutable": False,
                    }
                )

with open(OUTPUT, "w") as f:
    json.dump(result, f)

In [13]:
import re

names = []

with open('companies', 'r') as file:
    lines = file.readlines()

    for line in lines:
        ms = [
            re.search('(?<=\d\. )[\w \.]*', line),
            re.search('(?<=\d\d\. )[\w \.]*', line),
            re.search('(?<=\d\d\d\. )[\w \.]*', line),
        ]
        for m in ms:
            if m:
                names.append(f"'{m.group(0).strip()}'\n")

        names = list(set(names))

with open('companies_parsed', 'w') as file:
    file.writelines(names)
